In [1]:
!pip install torchtext==0.10.0 pydantic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
!pip install spacy

NotImplementedError: ignored

In [2]:
!python -m spacy download en_core_web_sm

2023-04-26 00:05:32.631061: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 00:05:33.893140: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-26 00:05:37.440592: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-26 00:05:37.441626: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

In [3]:
import spacy
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchtext
from torch.utils.data import Dataset
from torchtext.legacy.data import Iterator
from torchtext.legacy.data import Example
from torchtext.legacy.data import Field
from torchtext.datasets import SQuAD2
import random
import string

In [4]:
train_data, val_data = SQuAD2(split=('train', 'dev'))

In [5]:
train_dictionary = {"Questions" : [], "Answers": []}
for _, q, a, _ in train_data:
    train_dictionary["Questions"].append(q)
    train_dictionary["Answers"].append(a[0])
train_df = pd.DataFrame(train_dictionary)

In [6]:
val_dictionary = {"Questions" : [], "Answers": []}
for _, q, a, _ in val_data:
    val_dictionary["Questions"].append(q)
    val_dictionary["Answers"].append(a[0])
val_df = pd.DataFrame(val_dictionary)

In [7]:
train_df.head()

,Questions,Answers
0,When did Beyonce start becoming popular?,in the late 1990s
1,What areas did Beyonce compete in when she was...,singing and dancing
2,When did Beyonce leave Destiny's Child and bec...,2003
3,In what city and state did Beyonce grow up?,"Houston, Texas"
4,In which decade did Beyonce become famous?,late 1990s


In [8]:
val_df.head()

,Questions,Answers
0,In what country is Normandy located?,France
1,When were the Normans in Normandy?,10th and 11th centuries
2,From which countries did the Norse originate?,"Denmark, Iceland and Norway"
3,Who was the Norse leader?,Rollo
4,What century did the Normans first gain their ...,10th century


In [9]:
class Vocab:
    def __init__(self):
        self.word2index = {"SOS" : 0, "EOS": 1}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [10]:
nlp = spacy.load('en_core_web_sm')

def prepare_text(sentence):
    sentence = ''.join([s.lower() for s in sentence if s not in string.punctuation])
    tokens = ' '.join([token.text for token in nlp(sentence)])

    return tokens

def prepare_data(data_df):
    data_df['Questions'] = data_df['Questions'].apply(prepare_text)
    data_df['Answers'] = data_df['Answers'].apply(prepare_text)

    return data_df

In [11]:
train_df = prepare_data(train_df.iloc[:500, :])

<ipython-input-10-10a4e3c7965d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['Questions'] = data_df['Questions'].apply(prepare_text)
<ipython-input-10-10a4e3c7965d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['Answers'] = data_df['Answers'].apply(prepare_text)


In [12]:
def toTensor(vocab, sentence):
    indices = [vocab.word2index[word] for word in sentence.split(' ')]
    indices.append(vocab.word2index['EOS'])
    return torch.Tensor(indices).long().to(device).view(-1, 1)

def getPairs(df):
    temp1 = df["Questions"].apply(lambda x: "".join(x) ).to_list()
    temp2 = df["Answers"].apply(lambda x: "".join(x) ).to_list()
    return [list(i) for i in zip(temp1, temp2)]

def getMaxLen(pairs):
    max_src = 0 
    max_trg = 0
    
    for p in pairs:
        max_src = len(p[0].split()) if len(p[0].split()) > max_src else max_src
        max_trg = len(p[1].split()) if len(p[1].split()) > max_trg else max_trg
        
    return max_src, max_trg

In [13]:
train_pairs = getPairs(train_df)

In [14]:
train_pairs[:5]

[['when did beyonce start becoming popular', 'in the late 1990s'],
 ['what areas did beyonce compete in when she was growing up',
  'singing and dancing'],
 ['when did beyonce leave destinys child and become a solo singer', '2003'],
 ['in what city and state did beyonce   grow up', 'houston texas'],
 ['in which decade did beyonce become famous', 'late 1990s']]

In [15]:
Q_vocab = Vocab()
A_vocab = Vocab()

for pair in train_pairs:
    Q_vocab.addSentence(pair[0])
    A_vocab.addSentence(pair[1])

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
source_data = [toTensor(Q_vocab, pair[0]) for pair in train_pairs]
target_data = [toTensor(A_vocab, pair[1]) for pair in train_pairs]

In [18]:
max_src, max_trg = getMaxLen(train_pairs)

In [19]:
source_data[0]

tensor([[2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [1]], device='cuda:0')

In [20]:
target_data[0]

tensor([[2],
        [3],
        [4],
        [5],
        [1]], device='cuda:0')

In [21]:
Q_vocab.n_words

930

In [22]:
def evaluate(tensor, model):
    model.eval()
    batch_size = tensor.shape[1]
    outputs = []
    hidden, cell = model.encoder.init_hidden(batch_size)
    for i in range(tensor.shape[0]):
        _, hidden, cell = model.encoder(tensor[i], hidden, cell)
    x = torch.zeros(1, batch_size, dtype=torch.long, device=device)
    counts = 0
    while x != A_vocab.word2index["EOS"] and counts < max_trg + 20:
        output, hidden, cell = model.decoder(x, hidden, cell)
        best_guess = output.argmax(1)
        outputs.append(best_guess.item())
        x = best_guess.unsqueeze(0)
        counts += 1
    output_text = [A_vocab.index2word[idx] for idx in outputs]
    output_sentence = ' '.join(output_text)
    return output_sentence, outputs

In [23]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, num_layers, dropout=0.2):
        super().__init__()

        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.num_layers = num_layers
        self.dropout_prob = dropout

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.lstm = nn.LSTM(self.emb_dim, self.emb_dim, self.num_layers, dropout=self.dropout_prob, bidirectional=True)
        self.dropout_layer = nn.Dropout(self.dropout_prob)
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers*2, batch_size, self.emb_dim).to(device)
        cell = torch.zeros(self.num_layers*2, batch_size, self.emb_dim).to(device)
        return hidden, cell

    def forward(self, x, hidden, cell):
        x = x.unsqueeze(0)
        x = self.dropout_layer(self.embedding(x))
        x, (hidden, cell) = self.lstm(x, (hidden, cell))
        return x, hidden, cell
    
class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, num_layers, dropout=0.2):
        super().__init__()

        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout_prob = dropout

        self.embedding = nn.Embedding(output_dim, hidden_dim)
        self.lstm = nn.LSTM(self.hidden_dim, self.hidden_dim, self.num_layers, dropout=self.dropout_prob, bidirectional=True)
        self.fc = nn.Linear(self.hidden_dim*2, self.output_dim)
        self.dropout_layer = nn.Dropout(self.dropout_prob)
        
    def forward(self, x, hidden, cell):
        x = self.dropout_layer(self.embedding(x))
        x, (hidden, cell) = self.lstm(x, (hidden, cell))
        x = self.dropout_layer(self.fc(x.squeeze(0)))
        return x, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(device)

        hidden, cell = self.encoder.init_hidden(batch_size)
        
        for i in range(src.shape[0]):
            _, hidden, cell = encoder(src[i], hidden, cell)
            
        x = torch.zeros(1, batch_size, dtype=torch.long, device=device)
        
        for t in range(1, trg_len):
            output, hidden, cell = decoder(x, hidden, cell)
            outputs[t] = output
            best_guess = output.argmax(1)
            x = trg[t].unsqueeze(0) if random.random() < teacher_forcing_ratio else best_guess.unsqueeze(0)
        
        return outputs

In [24]:
input_dim = Q_vocab.n_words
output_dim = A_vocab.n_words
hidden_dim = 512
dropout = 0.4
batch_size = 16

In [25]:
output_dim

496

In [26]:
torch.autograd.set_detect_anomaly(True)

In [27]:
encoder = Encoder(input_dim, hidden_dim, 2, dropout).to(device)
decoder = Decoder(output_dim, hidden_dim, 2, dropout).to(device)
model = Seq2Seq(encoder, decoder).to(device)

In [28]:
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [29]:
sentence, output_tensor = evaluate(source_data[0], model)

In [30]:
print(sentence)
print(output_tensor)

run authors maiden maiden comedy darlette i i 20 myers myers paris drunk billboard out i 20 myers paris feminism stage stage concert 4 z hill second modernday second bonnie second
[333, 440, 50, 50, 152, 62, 121, 121, 47, 138, 138, 347, 332, 448, 159, 121, 47, 138, 347, 459, 263, 263, 467, 205, 23, 231, 358, 40, 358, 351, 358]


In [31]:
num_epochs = 10
total = 0
loss = 0
for epoch in range(num_epochs):
    model.train()
    loss = 0
    for i in range(len(source_data)):
        src = source_data[i].to(device)
        trg = target_data[i].to(device)
        output = model(src, trg)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        current = criterion(output, trg)
        loss += current
        total += current
        if (i + 1) % batch_size == 0 or i == (len(source_data)-1):
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loss = 0
            
    print_loss = total / len(source_data)
    print(f"Epoch {epoch + 1}/{num_epochs} | Training Loss: {print_loss:.3f}")
    total = 0
    if (epoch + 1) % 5 == 0:
      learning_rate = learning_rate / 10
      optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    model.eval()
    sentence, output_tensor = evaluate(source_data[0], model)
    print(output_tensor)
    print(sentence)
    print("\n\n\n")

        


Epoch 1/10 | Training Loss: 5.284
[1]
EOS




Epoch 2/10 | Training Loss: 4.700
[1]
EOS




Epoch 3/10 | Training Loss: 4.642
[1]
EOS




Epoch 4/10 | Training Loss: 4.521
[1]
EOS




Epoch 5/10 | Training Loss: 4.494
[1]
EOS




Epoch 6/10 | Training Loss: 4.441
[1]
EOS




Epoch 7/10 | Training Loss: 4.300
[1]
EOS




Epoch 8/10 | Training Loss: 4.147
[1]
EOS




Epoch 9/10 | Training Loss: 3.930
[1]
EOS




Epoch 10/10 | Training Loss: 4.027
[1]
EOS




